In [1]:
#importing used packages
import csv
from selenium import webdriver 
import pandas as pd
import numpy as np
import time
import re

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

## Function to get all products link with addition argument as number of pages want to scrap

In [3]:
def get_product_links(driver,number_of_pages,product_name):
    
    
    product_name = re.subn(" ","+",product_name)[0]  #link has no spaces so we are changing sapce with "+"     
    links=[]
    i=1
    
    #extracting product links from different webpages
    while(i<number_of_pages+1):
        time.sleep(0.5)
        
        #opening different webpages by changing product name and page number for link extraction
        url = "https://www.amazon.in/s?k={}&page=".format(product_name.lower())+str(i)+"&qid=1634839581&ref=sr_pg_1"
        driver.get(url) 
        time.sleep(0.5)
        
        #extracting all the link attached to products name
        all_a = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
        for a in all_a:
            links.append(a.get_attribute("href"))
        i+=1
        
    return links


## Function to get all features of the first product

In [4]:
def get_features_from_first_product(driver,link):
    driver.get(link)
    col = ['Name','Price']
    try:
            #extracting all the feactures(values in dark column)
            c = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
            
            #if table is not present extract data from the list style and send the variable to user
            if (len(c)==0):
                style = 0 
                c = driver.find_elements_by_xpath('//ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]//span[@class="a-text-bold"]')
                for x in c:
                    if x.text == "Best Sellers Rank:":
                        break
                    if 0 < len(x.text):
                        col.append(x.text[:-2])
            else:
                style = 1
                for x in c:
                    col.append(x.text)
                
    except:
            None
    return col,style

## Function to get data from amazon

In [15]:
def get_data_from_amazon(page_number):
    
    #get Product name
    product_name = input("Enter the product name: ")
    
    driver = webdriver.Chrome(PATH) #open browser
    
    link = get_product_links(driver,page_number,product_name) #extracting all products link
    print(len(link))
    a = np.arange(len(link))
    
    col,style = get_features_from_first_product(driver,link[0]) #extracting features
    product = pd.DataFrame(index = a,columns=col)
    print(col)
    
    #extracting information from all links
    for i in a:
            print(i)
            driver.get(link[i])
            time.sleep(0.5)
            
            #extracting product name
            try:
                name = driver.find_element_by_xpath('//*[@id="productTitle"]').text 
                product.loc[i,'Name'] = name
            except:
                None
                
            #extracting product price
            try:
                price = driver.find_element_by_xpath('//td[@class="a-span12"]/span[1]').text or driver.find_element_by_xpath('//*[@id="priceblock_dealprice"]').text   
                product.loc[i,'Price'] = price
            except:
                None
                
            try:
                if style == 0:
                    #extracting all the features of the current product on the webpage
                    c = driver.find_elements_by_xpath('//ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]//span[@class="a-text-bold"]') 
                    #extracting all the values corresponding to the features 
                    d = driver.find_elements_by_xpath('//ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]//span[@class="a-list-item"]/span[2]') 
                    #saving the values of the selected featues in the database 
                    for x in range(len(c)):
                        #print(x,c[x].text)
                        if(len(c[x].text)):
                            #print(len(c))
                            break
                    for j in range(len(d)):
                        if c[x+j].text[:-2] in col:
                            product.loc[i,c[x+j].text[:-2]] = d[j].text
                        else:
                            col.append(c[x+j].text[:-2])
                            product.loc[i,c[x+j].text[:-2]] = d[j].text
                else:
                    #extracting all the features of the current product on the webpage
                    c = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
                    #extracting all the values corresponding to the features 
                    d = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//td[@class="a-size-base prodDetAttrValue"]')
                    
                    #saving the values of the selected featues in the database
                    for x in range(len(c)):
                        if c[x].text in col:
                            product.loc[i,c[x].text] = d[x].text
            except:
                print(len(c)," ",len(d))

    driver.quit()
    #save the dataframe as same name as product_name 
    product.to_csv(product_name+'.csv')
    return product

In [19]:
product = get_data_from_amazon(20) #passing page number for easy test

Enter the product name: laptop
395
['Name', 'Price', 'Brand', 'Manufacturer', 'Series', 'Colour', 'Form Factor', 'Item Height', 'Item Width', 'Standing screen display size', 'Screen Resolution', 'Resolution', 'Product Dimensions', 'Batteries', 'Item model number', 'Processor Brand', 'Processor Type', 'Processor Speed', 'Processor Count', 'RAM Size', 'Memory Technology', 'Memory Clock Speed', 'Hard Drive Size', 'Hard Disk Description', 'Audio Details', 'Speaker Description', 'Graphics Coprocessor', 'Graphics Chipset Brand', 'Graphics Card Description', 'Graphics Card Ram Size', 'Connectivity Type', 'Number of USB 3.0 Ports', 'Number of Audio-out Ports', 'Voltage', 'Front Webcam Resolution', 'Power Source', 'Hardware Platform', 'Operating System', 'Average Battery Standby Life (in hours)', 'Average Battery Life (in hours)', 'Are Batteries Included', 'Lithium Battery Energy Content', 'Lithium Battery Weight', 'Number Of Lithium Ion Cells', 'Included Components', 'Manufacturer', 'Country o

In [20]:
product

,Name,Price,Brand,Manufacturer,Series,Colour,Form Factor,Item Height,Item Width,Standing screen display size,...,Average Battery Standby Life (in hours),Average Battery Life (in hours),Are Batteries Included,Lithium Battery Energy Content,Lithium Battery Weight,Number Of Lithium Ion Cells,Included Components,Manufacturer,Country of Origin,Item Weight
0,HP Chromebook 14-inch (35.56 cms) Thin & Light...,"₹26,990.00",HP,hp,Chromebook,Mineral Silver,Chromebook,18 Millimeters,32.6 Centimeters,14 Inches,...,12 Hours,12 Hours,Yes,47 Watt Hours,0.85 Grams,2,"Chromebook, Battery, AC Adapter, User Guide, M...",hp,China,1 kg 460 g
1,"HP 15 11th Gen Intel Core i3 Processor 15.6"" (...","₹42,990.00",HP,hp,NaN,NaN,Laptop,NaN,NaN,15.6 Inches,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hp,China,1 kg 770 g
2,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...","₹30,900.00",ASUS,Asus,ASUS VivoBook 15 (2020),Transparent Silver,Ultra-Portable,20 Millimeters,36 Centimeters,15.6 Inches,...,6 Hours,6 Hours,Yes,37 Watt Hours,NaN,2,"Laptop, AC Adapter, User Guide, Manuals",Asus,China,1 kg 800 g
3,Lenovo IdeaPad Slim 1 Intel Celeron N4020 11.6...,"₹25,990.00",Lenovo,Lenovo,NaN,NaN,Thin & Light,NaN,NaN,11.6 Inches,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lenovo,China,1 kg 200 g
4,HP Chromebook 14-inch (35.56 cms) Thin & Light...,"₹26,990.00",HP,hp,Chromebook,Mineral Silver,Chromebook,18 Millimeters,32.6 Centimeters,14 Inches,...,12 Hours,12 Hours,Yes,47 Watt Hours,0.85 Grams,2,"Chromebook, Battery, AC Adapter, User Guide, M...",hp,China,1 kg 460 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,(Renewed) Dell Intel Core i3 4th Gen 14-Inch (...,"₹33,999.00",Dell,Dell Computers,Latitude-cr,Black,Laptop,0.7 Inches,13.3 Inches,12.5 Inches,...,NaN,3 Hours,NaN,NaN,NaN,NaN,Laptop.adaptor.powercard,Dell Computers,India,1 kg 500 g
391,HP 15 11th Gen Intel Core i3 Processor 15.6 in...,"₹45,999.00",HP,hp,NaN,NaN,Laptop,1.79 Centimeters,24.2 Centimeters,15.6 Inches,...,6 Hours,4 Hours,Yes,41 Watt Hours,0.85 Grams,NaN,NaN,hp,China,1 kg 690 g
392,Fujitsu UH-X 11th Gen Intel Core i5 13.3” (33....,"₹59,990.00",Fujitsu,FUJITSU,UH-X Thin and Light,Black,Laptop,16 Millimeters,19.7 Centimeters,13.3 Inches,...,11 Hours,10 Hours,Yes,NaN,NaN,4,"Laptop Computer System, Adapter, Power Cord, M...",FUJITSU,Japan,878 g
393,(Renewed) HP Intel Core i7 6700HQ 15.6-Inch (3...,"₹82,490.00",HP,hp,NaN,Grey,Laptop,25 Millimeters,26.4 Centimeters,15.6 Inches,...,NaN,NaN,Yes,44 Watt Hours,NaN,NaN,NaN,hp,NaN,2 kg 700 g


In [22]:
(product.isnull().sum()/395)*100

Name                                        0.000000
Price                                       4.556962
Brand                                       0.253165
Manufacturer                                1.772152
Series                                     27.594937
Colour                                     27.088608
Form Factor                                 6.329114
Item Height                                21.265823
Item Width                                 21.265823
Standing screen display size                1.518987
Screen Resolution                          79.746835
Resolution                                  4.556962
Product Dimensions                          1.772152
Batteries                                  12.151899
Item model number                          21.265823
Processor Brand                             1.772152
Processor Type                              3.037975
Processor Speed                             4.810127
Processor Count                             2.